In [30]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("[,;\s]\s*", text, flags=re.UNICODE)
    for word in words:
        flag = False
        for w in list(word):
            if (w <= 'z' and w >= 'a') or (w <= 'Z' and w >= 'A'):
                flag = True
            else:
                flag = False
                break
        if flag is True:
            print(sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1)
            print("%s\t%d" % (word.lower(), 1))

Overwriting mapper.py


In [23]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [24]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print("%s\t%d" % (current_key, word_sum))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print("%s\t%d" % (current_key, word_sum))

Appending to reducer.py


In [25]:
%%writefile mapper_rating.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
        
    print "%d\t%s" % (count, word)

Overwriting mapper_rating.py


In [26]:
%%writefile reducer_rating.py

import sys

key_words = [-1 for i in range(26)]
num_counts = [0 for i in range(26)]

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if count > num_counts[ord(word[0]) - ord('a')]:
        key_words[ord(word[0]) - ord('a')] = word
        num_counts[ord(word[0]) - ord('a')] = count

for i in range(26):
    print "%s\t%d" % (key_words[i], num_counts[i])

Overwriting reducer_rating.py


In [29]:
%%bash

OUT_DIR="wordcount_result_for_rating"
NUM_REDUCERS=8

# 删除文件
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR}/ > /dev/null


OUT_DIR_2="wordcount_result_for_rating_2"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR_2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount Rating" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D map.output.key.field.separator=\t \
    -D mapreduce.partition.keycomparator.options=-k1,1nr \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper_rating.py,reducer_rating.py \
    -mapper "python mapper_rating.py" \
    -reducer "python reducer_rating.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR_2} > /dev/null

hdfs dfs -cat ${OUT_DIR_2}/part-00000 | head -26

and	341101
by	85253
can	18036
during	12330
early	8311
for	89827
government	6052
his	44810
in	289407
john	5518
known	9906
later	8120
more	18107
not	29821
of	446789
part	7614
quickly	1024
references	4074
such	18572
the	812210
used	17510
very	5273
was	90134
x	2675
years	8534
zealand	719


22/06/27 17:13:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
22/06/27 17:13:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
22/06/27 17:13:15 INFO mapred.FileInputFormat: Total input files to process : 1
22/06/27 17:13:15 INFO mapreduce.JobSubmitter: number of splits:2
22/06/27 17:13:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1656346777937_0014
22/06/27 17:13:15 INFO impl.YarnClientImpl: Submitted application application_1656346777937_0014
22/06/27 17:13:15 INFO mapreduce.Job: The url to track the job: http://648416f5b978:8088/proxy/application_1656346777937_0014/
22/06/27 17:13:15 INFO mapreduce.Job: Running job: job_1656346777937_0014
22/06/27 17:13:20 INFO mapreduce.Job: Job job_1656346777937_0014 running in uber mode : false
22/06/27 17:13:20 INFO mapreduce.Job:  map 0% reduce 0%
22/06/27 17:13:36 INFO mapreduce.Job:  map 17% reduce 0%
22/06/27 17:13:42 INFO mapreduce.Job:  map 25% reduce 0%
22/06/27 17:13:47 INFO 

In [21]:
! hdfs dfs -cat wordcount_result_for_rating/part-00000 | head -26

a	230878
aaa	32
aaba	2
aabsbaa	1
aacca	3
aafedt	2
aagaard	2
aai	1
aakaar	1
aallon	1
aals	1
aardappeloproer	1
aarne	1
aaron	272
aaronids	2
aaronow	7
aasmund	2
aave	2
ababda	4
abacuses	1
abadal	1
abadeh	16
abandonmentwith	1
abano	7
abasciano	3
abate	6
cat: Unable to write to output stream.
